In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch
from torch import from_numpy, cuda
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
from torchvision import transforms
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR, SequentialLR, LinearLR
from PIL import Image
from torch.cuda.amp import GradScaler, autocast

In [2]:
device = torch.device("cuda" if cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
with open("/kaggle/input/food-101/food-101/food-101/meta/classes.txt", 'r') as f:
    classes = f.read().splitlines()
len(classes)

101

In [4]:
class Label_encoder:
    def __init__(self, labels):
        self.labels = {label: idx for idx, label in enumerate(labels)}
    def get_label(self, idx):
        return list(self.labels.keys())[idx]
    def get_idx(self, label):
        return self.labels.get(label)

encoder = Label_encoder(classes)

In [5]:
class Food101(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.path.iloc[idx]
        image = Image.open(img_name)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        label = encoder.get_idx(self.dataframe.label.iloc[idx])

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
def prep_df(path: str) -> pd.DataFrame:
    with open(path, 'r') as f:
        names = f.read().splitlines()
    img_path = "/kaggle/input/food-101/food-101/food-101/images/"
    full_path = [img_path + img + ".jpg" for img in names]
    imgs = []
    for img in names:
        img = img.split('/')
        imgs.append(img)
    imgs = np.array(imgs)
    
    df = pd.DataFrame(imgs[:, 0], imgs[:,1], columns=['label'])      
    df['path'] = full_path
    df = shuffle(df)
    return df

train_imgs = prep_df('/kaggle/input/food-101/food-101/food-101/meta/train.txt')
test_imgs = prep_df('/kaggle/input/food-101/food-101/food-101/meta/test.txt')

train_imgs.groupby(["label"]).count().reset_index()

,label,path
0,apple_pie,750
1,baby_back_ribs,750
2,baklava,750
3,beef_carpaccio,750
4,beef_tartare,750
...,...,...
96,tacos,750
97,takoyaki,750
98,tiramisu,750
99,tuna_tartare,750


In [7]:
class CutOut(object):
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)

        for _ in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)
            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)
            mask[y1: y2, x1: x2] = 0.

        mask = from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask
        return img

In [8]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    CutOut(n_holes=1, length=16)
])


transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [9]:
trainset = Food101(train_imgs, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

testset = Food101(test_imgs, transform=transform_test)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [10]:
class ImprovedFoodCNN(nn.Module):
    def __init__(self, num_classes=101):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.1),
            
            # Второй блок
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2),
            
            # Третий блок
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3),
            
            # Четвертый блок
            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.3),
            nn.Linear(1024, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [11]:
model = ImprovedFoodCNN().to(device)

In [12]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)

warmup_epochs = 3
total_epochs = 30

scheduler = SequentialLR(
    optimizer,
    schedulers=[
        LinearLR(optimizer, start_factor=0.1, total_iters=warmup_epochs),
        CosineAnnealingLR(optimizer, T_max=total_epochs - warmup_epochs)
    ],
    milestones=[warmup_epochs]
)


In [13]:
def train(epoch):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    loop = tqdm(trainloader, desc=f"Training Epoch {epoch + 1}", leave=False)

    for inputs, targets in loop:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
    
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += targets.size(0)
        correct_train += predicted.eq(targets).sum().item()

        loop.set_postfix(loss=loss.item())

    train_accuracy = 100. * correct_train / total_train
    avg_loss = running_loss / len(trainloader)
    print(f"Epoch {epoch+1}: Train Loss = {avg_loss:.4f}, Accuracy = {train_accuracy:.2f}%")

# --- Тестирование ---
def test(epoch):
    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, targets in tqdm(testloader, desc=f"Testing Epoch {epoch + 1}", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_test += targets.size(0)
            correct_test += predicted.eq(targets).sum().item()

    test_accuracy = 100. * correct_test / total_test
    print(f"Test Accuracy after Epoch {epoch + 1}: {test_accuracy:.2f}%")
    return test_accuracy

In [14]:
torch.save(model.state_dict(), "/kaggle/working/food101_best.pt")

In [ ]:
best_acc = 0.0
print("Starting training...\n")

for epoch in range(total_epochs):
    train(epoch)
    acc = test(epoch)
    scheduler.step()

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "/kaggle/working/food101_best.pt")
        print(f"✅ Best model saved with accuracy: {best_acc:.2f}%")
    if epoch % 5 == 0:
        checkpoint = {
            'epoch': total_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'rng_state': torch.get_rng_state()
        }
    
        torch.save(checkpoint, f'/kaggle/working/model_epoch_{epoch}.pth')
        torch.save(checkpoint, f'/kaggle/working/model_epoch_{epoch}.tar')
print("\n🎉 Training completed.")

Starting training...



Epoch 1: Train Loss = 4.6048, Accuracy = 2.88%


Test Accuracy after Epoch 1: 7.78%
✅ Best model saved with accuracy: 7.78%


Epoch 2: Train Loss = 4.5110, Accuracy = 3.94%


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

Epoch 5: Train Loss = 4.2480, Accuracy = 7.47%


Test Accuracy after Epoch 5: 15.20%
✅ Best model saved with accuracy: 15.20%


Epoch 6: Train Loss = 4.1602, Accuracy = 9.02%


Test Accuracy after Epoch 6: 17.76%
✅ Best model saved with accuracy: 17.76%


Training Epoch 7:  63%|██████▎   | 1483/2368 [13:29<08:10,  1.80it/s, loss=3.91]